提取单个nii文件

In [ ]:
# 提取单个nii文件
import os
import SimpleITK as sitk
import pydicom
import re

def is_valid_dicom(file_path):
    """检查文件是否为有效的DICOM文件"""
    try:
        pydicom.dcmread(file_path, stop_before_pixels=True)
        return True
    except:
        return False

def get_dicom_series_desc(file_path):
    """仅获取DICOM文件的序列描述（无需患者名，按原始文件名命名）"""
    try:
        ds = pydicom.dcmread(file_path, stop_before_pixels=True)
        return ds.SeriesDescription if hasattr(ds, 'SeriesDescription') else ""
    except Exception as e:
        print(f"⚠️ 读取DICOM序列描述失败 {os.path.basename(file_path)}: {str(e)}")
        return ""

def get_existing_files(output_dir):
    """扫描输出目录，获取已处理的文件（基于原始文件名_front.nii.gz）"""
    existing_original_names = set()
    if not os.path.exists(output_dir):
        return existing_original_names
    
    # 遍历输出目录中所有_front.nii.gz文件
    for filename in os.listdir(output_dir):
        if filename.endswith("_front.nii.gz"):
            # 提取原始文件名（去掉 "_front.nii.gz" 后缀）
            original_name = filename.replace("_front.nii.gz", "")
            existing_original_names.add(original_name)
    
    print(f"ℹ️ 已发现 {len(existing_original_names)} 个已处理文件（基于原始文件名）")
    return existing_original_names

def export_wb_series(input_dir, output_root, wb_keyword="WB"):
    """适配单个DICOM文件场景，按「原始文件名_front.nii.gz」导出，支持断点续传"""
    # 检查输入路径
    if not os.path.exists(input_dir):
        print(f"错误：输入路径不存在 - {input_dir}")
        return
    if not os.path.isdir(input_dir):
        print(f"错误：输入路径不是文件夹 - {input_dir}")
        return
        
    os.makedirs(output_root, exist_ok=True)
    print(f"输出文件将保存到：{output_root}")
    print(f"文件命名格式：原始文件名_front.nii.gz")
    print(f"正在扫描文件夹中的单个DICOM文件：{input_dir}\n")
    
    # 关键修改1：获取已处理的原始文件名（断点续传核心）
    processed_original_names = get_existing_files(output_root)
    valid_dicom_count = 0       # 有效DICOM文件计数
    newly_processed = 0         # 本次新增处理的数量
    
    # 遍历文件夹中所有文件
    for filename in os.listdir(input_dir):
        file_path = os.path.join(input_dir, filename)
        # 跳过文件夹，只处理文件
        if not os.path.isfile(file_path):
            continue
        
        # 跳过非DICOM文件（仅保留.dcm后缀）
        if not filename.lower().endswith('.dcm'):
            continue
        
        # 检查是否为有效DICOM
        if not is_valid_dicom(file_path):
            print(f"ℹ️ 跳过无效DICOM文件：{filename}")
            continue
        
        valid_dicom_count += 1
        series_desc = get_dicom_series_desc(file_path)
        
        # 关键修改2：提取原始文件名（不含.dcm后缀），处理特殊字符确保保存合法
        original_name_without_ext = os.path.splitext(filename)[0]
        # 替换Windows非法字符（\ / : * ? " < > |）为下划线，避免保存失败
        safe_original_name = re.sub(r'[\\/:*?"<>|]', '_', original_name_without_ext)
        
        # 关键修改3：基于原始文件名判断是否已处理（断点续传）
        if safe_original_name in processed_original_names:
            print(f"ℹ️ 文件已处理，跳过：{filename} → 对应输出：{safe_original_name}_front.nii.gz")
            continue
        
        # 筛选WB序列（不区分大小写）
        if wb_keyword.upper() not in (series_desc or "").upper():
            print(f"ℹ️ 跳过非WB序列 - 文件名：{filename} | 序列描述：{series_desc}")
            continue
        
        try:
            # 读取单个DICOM文件
            image = sitk.ReadImage(file_path)
            
            # 关键修改4：按「原始文件名_front.nii.gz」保存
            output_filename = os.path.join(output_root, f"{safe_original_name}_front.nii.gz")
            
            # 保存为nii.gz
            sitk.WriteImage(image, output_filename)
            print(f"✅ 已导出 - 原始文件：{filename} | WB序列：{series_desc} → 输出：{os.path.basename(output_filename)}")
            
            # 标记为已处理（避免本次运行重复处理）
            processed_original_names.add(safe_original_name)
            newly_processed += 1
            
        except Exception as e:
            print(f"❌ 处理失败 - 原始文件：{filename} | 错误：{str(e)}")
    
    # 输出统计信息
    print(f"\n{'='*50}")
    print(f"扫描完成：共发现 {valid_dicom_count} 个有效DICOM文件")
    print(f"已处理（历史+本次）：{len(processed_original_names)} 个WB序列文件")
    print(f"本次新增处理：{newly_processed} 个WB序列文件")
    print(f"输出路径：{output_root}")
    print(f"文件命名格式：原始文件名_front.nii.gz")
    print(f"{'='*50}")

if __name__ == "__main__":
    # 配置路径（所有单个DICOM文件所在的文件夹）
    INPUT_DIRECTORY = r"/home/wangnannan/data/spect/sz/all_images"
    OUTPUT_DIRECTORY = r"/home/gonghanmei/datasets/shen_data/only_wb_all_x"
    WB_KEYWORD = "WB"  # 可调整为实际WB序列描述（如"Whole Body"、"全身"等）
    
    export_wb_series(INPUT_DIRECTORY, OUTPUT_DIRECTORY, WB_KEYWORD)
    print("\n批量处理完成！")


In [ ]:
# # 5单纯处理nii to png
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import rotate

def nii_to_png(nii_file_path, output_dir, mirror=True):
    # 读取nii文件
    img = nib.load(nii_file_path)
    data = img.get_fdata()
    
    # 获取文件名（不含扩展名）
    file_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file_path))[0])[0]
    
    # 修正：back和front对调，并添加顺时针旋转90°
    # 正面切片保存（原反面）
    front_slice = data[:, :, 0]  # 原反面切片
    # 顺时针旋转90度（使用-90表示顺时针）
    front_rotated = rotate(front_slice, -90, reshape=True)
    
    # 如果需要镜像处理，进行左右翻转
    if mirror:
        front_rotated = np.fliplr(front_rotated)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(front_rotated, cmap='gray')
    plt.axis('off')
    front_output_path = os.path.join(output_dir, f"{file_name}_front.png")
    plt.savefig(front_output_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # 反面切片保存（原正面）
    back_slice = data[:, :, 1]  # 原正面切片
    # 顺时针旋转90度
    back_rotated = rotate(back_slice, -90, reshape=True)
    
    # 如果需要镜像处理，进行左右翻转
    if mirror:
        back_rotated = np.fliplr(back_rotated)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(back_rotated, cmap='gray')
    plt.axis('off')
    back_output_path = os.path.join(output_dir, f"{file_name}_back.png")
    plt.savefig(back_output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def batch_convert(nii_dir, output_dir, mirror=True):
    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 遍历目录下的所有nii.gz文件
    for file in os.listdir(nii_dir):
        if file.endswith('.nii.gz'):
            nii_file_path = os.path.join(nii_dir, file)
            nii_to_png(nii_file_path, output_dir, mirror)
            print(f"已处理：{file}")



# 使用示例
if __name__ == "__main__":
    # 替换为你的nii文件所在文件夹路径
    nii_directory = r"/home/wangnannan/data/spect/sz/all_images_denoise"
    # 替换为你希望保存png图片的文件夹路径
    output_directory = r"/home/gonghanmei/datasets/shen_data/all_data/png-all-qz"
    
    # 可以通过设置mirror参数控制是否镜像，默认为True
    batch_convert(nii_directory, output_directory, mirror=True)
    print("批量转换完成！")

In [ ]:
# 1提取所有csv文件
import os
import pandas as pd
from glob import glob

def merge_csv_files():
    # 获取bone-data文件夹的路径
    bone_data_path = r"/home/DataCollection/Raw/SPECT/SZ_Cancer/W8-bone 62"
    
    # 检查bone-data文件夹是否存在
    if not os.path.exists(bone_data_path):
        print(f"错误：未找到文件夹 {bone_data_path}")
        return
    
    # 获取bone-data的上级目录
    parent_dir = os.path.dirname(bone_data_path)
    
    # 新建保存合并结果的文件夹（在bone-data同级）
    output_dir = os.path.join(parent_dir, "csv")
    os.makedirs(output_dir, exist_ok=True)
    print(f"合并后的文件将保存到：{output_dir}")
    
    # 遍历bone-data下的所有直接子目录
    for root, dirs, files in os.walk(bone_data_path):
        if root == bone_data_path:  # 只处理直接子目录
            for subdir in dirs:
                subdir_path = os.path.join(root, subdir)
                print(f"\n正在处理目录：{subdir_path}")
                
                # 查找该目录下所有CSV文件（按文件名排序，确保顺序一致）
                csv_files = sorted(glob(os.path.join(subdir_path, "*.csv")))
                
                # 准备输出文件路径
                output_file = os.path.join(output_dir, f"{subdir}.csv")
                
                if not csv_files:
                    print(f"  警告：该目录下未找到CSV文件，将生成空文件")
                    pd.DataFrame().to_csv(output_file, index=False)
                    print(f"  已生成空文件：{os.path.basename(output_file)}")
                    continue
                
                # 合并所有CSV文件，保持第一列分组递增
                dfs = []
                last_group = 0  # 记录上一个文件的最大分组号
                first_col_name = None  # 第一列的列名
                
                for i, csv_file in enumerate(csv_files):
                    try:
                        df = pd.read_csv(csv_file)
                        if df.empty:
                            print(f"  警告：{os.path.basename(csv_file)} 为空文件，跳过")
                            continue
                        
                        # 获取第一列列名（首次读取时确定）
                        if first_col_name is None:
                            first_col_name = df.columns[0]
                            print(f"  检测到第一列列名为：{first_col_name}（将保持该列分组递增）")
                        
                        # 检查当前文件是否有第一列
                        if first_col_name not in df.columns:
                            print(f"  警告：{os.path.basename(csv_file)} 缺少第一列 {first_col_name}，跳过")
                            continue
                        
                        # 提取当前文件的第一列数据
                        current_group_col = df[first_col_name]
                        
                        # 处理第一个文件：以其自身的分组为基准
                        if i == 0:
                            # 确保第一列是数值类型
                            if not pd.api.types.is_numeric_dtype(current_group_col):
                                print(f"  警告：{os.path.basename(csv_file)} 的第一列不是数值类型，无法处理，跳过")
                                continue
                            # 更新最后一个分组号
                            last_group = current_group_col.max()
                            dfs.append(df)
                            print(f"  已读取：{os.path.basename(csv_file)}（最大分组：{last_group}）")
                        else:
                            # 非第一个文件：将其分组号整体偏移，确保从 last_group + 1 开始
                            # 计算当前文件的分组偏移量（当前最小分组应该对应 last_group + 1）
                            current_min = current_group_col.min()
                            offset = (last_group + 1) - current_min
                            
                            # 偏移当前文件的第一列（保证分组连续递增）
                            df[first_col_name] = current_group_col + offset
                            
                            # 更新最后一个分组号
                            last_group = df[first_col_name].max()
                            dfs.append(df)
                            print(f"  已读取：{os.path.basename(csv_file)}（偏移后最大分组：{last_group}）")
                            
                    except Exception as e:
                        print(f"  读取失败 {os.path.basename(csv_file)}：{str(e)}")
                
                if not dfs:
                    print(f"  警告：该目录下所有CSV文件读取失败，将生成空文件")
                    pd.DataFrame().to_csv(output_file, index=False)
                    print(f"  已生成空文件：{os.path.basename(output_file)}")
                    continue
                
                # 合并所有数据框（保持原有顺序）
                merged_df = pd.concat(dfs, ignore_index=True)
                
                # 保存合并后的文件
                merged_df.to_csv(output_file, index=False)
                print(f"  合并完成，已保存到：{os.path.basename(output_file)}（最终最大分组：{last_group}）")
    
    print("\n所有目录处理完毕！")

if __name__ == "__main__":
    merge_csv_files()


In [ ]:

# 3转化为txt
import pandas as pd
import os
from pathlib import Path

def get_label(df, name: str, output_dir: str = None):
    """
    将CSV数据按照axis-0的值拆分为front(0)和back(1)两个txt文件
    即使没有数据也会生成空文件
    """
    # 设置输出目录
    labels_path = Path(output_dir) if output_dir else Path('.')
    labels_path.mkdir(parents=True, exist_ok=True)
    
    filter_column = 'axis-0'
    value_for_front = 0
    value_for_back = 1
    
    # 处理 front (axis-0 = 0)
    print(f"处理 front 数据 (axis-0 = {value_for_front})...")
    result = []
    
    # 处理空DataFrame情况
    if not df.empty and filter_column in df.columns:
        front_df = df[df[filter_column] == value_for_front]
        front_list = list(zip(front_df.get('axis-2', []).tolist(), 
                             front_df.get('axis-1', []).tolist()))[::2]
        
        # 按对处理坐标
        for i in range(0, len(front_list), 2):
            if i + 1 < len(front_list):
                coord1 = front_list[i]
                coord2 = front_list[i + 1]
                
                x_center = (coord1[0] + coord2[0]) / 2 / 256
                y_center = (coord1[1] + coord2[1]) / 2 / 1024
                width = (coord2[0] - coord1[0] + 40) / 2 / 256
                height = (coord2[1] - coord1[1] + 40) / 2 / 1024
                
                result.append((0, x_center, y_center, width, height))
    
    # 写入front文件（确保生成）
    front_file_name = labels_path / (name + '_front.txt')
    print(f"📝 正在写入front文件: {front_file_name}")
    with open(front_file_name, 'w', encoding='utf-8') as f:
        for sublist in result:
            line = ' '.join(map(str, sublist))
            f.write(line + '\n')
    
    print(f"✅ Front文件已保存: {front_file_name}")
    print(f"   文件大小: {os.path.getsize(front_file_name)} 字节")
    print(f"   包含 {len(result)} 个标签")
    
    # 处理 back (axis-0 = 1)
    print(f"\n处理 back 数据 (axis-0 = {value_for_back})...")
    result = []
    
    # 处理空DataFrame情况
    if not df.empty and filter_column in df.columns:
        back_df = df[df[filter_column] == value_for_back]
        back_list = list(zip(back_df.get('axis-2', []).tolist(), 
                            back_df.get('axis-1', []).tolist()))[::2]
        
        # 按对处理坐标
        for i in range(0, len(back_list), 2):
            if i + 1 < len(back_list):
                coord1 = back_list[i]
                coord2 = back_list[i + 1]
                
                x_center = (coord1[0] + coord2[0]) / 2 / 256
                y_center = (coord1[1] + coord2[1]) / 2 / 1024
                width = (coord2[0] - coord1[0] + 40) / 2 / 256
                height = (coord2[1] - coord1[1] + 40) / 2 / 1024  # back使用40而不是20
                
                result.append((0, x_center, y_center, width, height))
    
    # 写入back文件（确保生成）
    back_file_name = labels_path / (name + '_back.txt')  # 保留原始名称中的空格
    print(f"📝 正在写入back文件: {back_file_name}")
    with open(back_file_name, 'w', encoding='utf-8') as f:
        for sublist in result:
            line = ' '.join(map(str, sublist))
            f.write(line + '\n')
    
    print(f"✅ Back文件已保存: {back_file_name}")
    print(f"   📄 绝对路径: {os.path.abspath(back_file_name)}")
    print(f"   📦 文件大小: {os.path.getsize(back_file_name)} 字节")
    print(f"   📊 包含 {len(result)} 个标签")

def process_single_file(csv_path, output_dir):
    """处理单个CSV文件，增强了对空文件和异常的处理"""
    try:
        print(f"\n📖 正在处理: {os.path.basename(csv_path)}")
        df = pd.read_csv(csv_path)
        
        print(f"   📊 数据信息: {len(df)} 行, {len(df.columns)} 列")
        
        # 检查必要的列是否存在（空文件允许缺少列）
        required_columns = ['axis-0', 'axis-1', 'axis-2']
        if not df.empty:
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                print(f"   ❌ 跳过: 缺少必要的列 {missing_columns}")
                return False
        
        # 显示axis-0的值分布
        if not df.empty and 'axis-0' in df.columns:
            axis_0_counts = df['axis-0'].value_counts().sort_index()
            print(f"   📈 axis-0 分布: ", end="")
            for value, count in axis_0_counts.items():
                print(f"{value}:{count}个 ", end="")
            print()
        else:
            print("   📈 axis-0 分布: 无数据")
        
        # 提取文件名（不含扩展名）
        file_name = os.path.splitext(os.path.basename(csv_path))[0]
        
        # 调用处理函数 - 即使没有数据也会生成文件
        get_label(df, file_name, output_dir)
        
        return True
        
    except Exception as e:
        print(f"   ❌ 处理失败: {e}")
        # 处理失败时也生成空文件
        file_name = os.path.splitext(os.path.basename(csv_path))[0]
        get_label(pd.DataFrame(), file_name, output_dir)
        return True  # 视为成功，因为生成了空文件

def main():
    # CSV文件夹路径
    csv_folder = r"/home/gonghanmei/datasets/sz_data_all/merged_folder"
    
    # 创建输出文件夹
    output_folder = os.path.join(csv_folder, "processed_labels")
    
    print(f"🔍 扫描文件夹: {csv_folder}")
    print(f"📁 输出文件夹: {output_folder}")
    
    # 创建输出目录
    os.makedirs(output_folder, exist_ok=True)
    print(f"✅ 输出文件夹已创建: {output_folder}")
    
    # 检查文件夹是否存在
    if not os.path.exists(csv_folder):
        print(f"❌ 错误: 找不到文件夹 {csv_folder}")
        print("请检查文件夹路径是否正确")
        return
    
    # 查找所有CSV文件
    csv_files = []
    for file in os.listdir(csv_folder):
        if file.lower().endswith('.csv'):
            csv_files.append(os.path.join(csv_folder, file))
    
    if not csv_files:
        print(f"❌ 在文件夹中没有找到CSV文件")
        print(f"📁 文件夹内容:")
        for file in os.listdir(csv_folder):
            print(f"   - {file}")
        return
    
    print(f"📋 找到 {len(csv_files)} 个CSV文件:")
    for i, file in enumerate(csv_files, 1):
        print(f"   {i}. {os.path.basename(file)}")
    
    print(f"\n🔄 开始批量处理...")
    
    success_count = 0
    total_count = len(csv_files)
    
    for csv_file in csv_files:
        if process_single_file(csv_file, output_folder):
            success_count += 1
    
    print(f"\n🎉 批量处理完成！")
    print(f"📊 统计: {success_count}/{total_count} 个文件处理成功")
    
    if success_count > 0:
        print(f"\n📁 所有输出文件都保存在: {output_folder}")
        print("生成的文件格式: 文件名_front.txt 和 文件名_back.txt")
        
        # 显示生成的文件列表
        print(f"\n📝 生成的文件列表:")
        txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]
        txt_files.sort()
        for i, txt_file in enumerate(txt_files, 1):
            file_size = os.path.getsize(os.path.join(output_folder, txt_file))
            print(f"   {i}. {txt_file} ({file_size} 字节)")

if __name__ == "__main__":
    main()


In [ ]:
# 6可视化普通可视化标签+图像
import os
import cv2
import numpy as np

def visualize_yolo_labels(image_dir, label_dir, output_dir, class_names=None):
    """
    可视化YOLO格式的标签到对应的图片上
    
    参数:
        image_dir: 图片文件夹路径
        label_dir: 标签文件夹路径
        output_dir: 输出可视化结果的文件夹路径
        class_names: 类别名称列表，如果没有则只显示类别ID
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 获取所有图片文件
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = [f for f in os.listdir(image_dir) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    for image_file in image_files:
        # 获取图片路径和文件名（不含扩展名）
        image_path = os.path.join(image_dir, image_file)
        base_name = os.path.splitext(image_file)[0]
        
        # 对应的标签文件路径
        label_file = os.path.join(label_dir, f"{base_name}.txt")
        
        # 检查标签文件是否存在
        if not os.path.exists(label_file):
            print(f"警告: 未找到 {image_file} 对应的标签文件 {label_file}")
            continue
        
        # 读取图片
        image = cv2.imread(image_path)
        if image is None:
            print(f"警告: 无法读取图片 {image_file}")
            continue
        
        height, width = image.shape[:2]
        
        # 读取并解析标签文件
        with open(label_file, 'r') as f:
            lines = f.readlines()
        
        # 绘制每个目标的边界框
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # YOLO格式: class_id center_x center_y width height (均为归一化值)
            parts = line.split()
            if len(parts) != 5:
                print(f"警告: 标签文件 {label_file} 中格式错误: {line}")
                continue
            
            class_id = int(parts[0])
            center_x = float(parts[1])
            center_y = float(parts[2])
            bbox_width = float(parts[3])
            bbox_height = float(parts[4])
            
            # 转换为绝对坐标
            x = int((center_x - bbox_width / 2) * width)
            y = int((center_y - bbox_height / 2) * height)
            w = int(bbox_width * width)
            h = int(bbox_height * height)
            
            # 绘制边界框 - 修改为红色且线条更细
            # color = (0, 0, 255)  # 红色 (BGR格式，所以是0,0,255)
            color = (255, 0, 0) 
            thickness = 1  # 线条粗细，改为1使边框更细
            cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
            
            # 绘制类别信息
            if class_names and class_id < len(class_names):
                label = f"{class_names[class_id]}"
            else:
                label = f"Class {class_id}"
            
            # # 放置标签文本（使用红色）
            # cv2.putText(image, label, (x, y - 10), 
            #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)  # 也减小了字体和线条粗细
        
        # 保存可视化结果
        output_path = os.path.join(output_dir, image_file)
        cv2.imwrite(output_path, image)
        print(f"已处理: {image_file} -> 保存到 {output_path}")

if __name__ == "__main__":
    # 配置路径（请根据实际情况修改）
    IMAGE_DIR = r"/home/gonghanmei/datasets/shen_data/all_data/png-all-jq"   # 图片所在文件夹
    LABEL_DIR = r"/home/gonghanmei/datasets/shen_data/all_data/pred_label"# 标签所在文件夹
    # LABEL_DIR = r"/home/gonghanmei/datasets/sz_data_all/merged_folder/processed_labels"
    OUTPUT_DIR = r"/home/gonghanmei/datasets/shen_data/all_data/png-all-jq/vis" # 可视化结果输出文件夹
    
    # 类别名称列表，如果有的话
    # 例如: CLASS_NAMES = ["class1", "class2", "class3"]
    CLASS_NAMES = None  # 如果没有类别名称列表，设为None
    
    # 执行可视化
    visualize_yolo_labels(IMAGE_DIR, LABEL_DIR, OUTPUT_DIR, CLASS_NAMES)
    print("所有图片处理完成！")


In [ ]:
# 6可视化+虚线
import os
import cv2
import numpy as np

def visualize_yolo_labels(image_dir, label_dir, output_dir, class_names=None, use_dashed_box=False):
    """
    可视化YOLO格式的标签到对应的图片上
    
    参数:
        image_dir: 图片文件夹路径
        label_dir: 标签文件夹路径
        output_dir: 输出可视化结果的文件夹路径
        class_names: 类别名称列表，如果没有则只显示类别ID
        use_dashed_box: 是否使用虚线框，默认为False（实线框）
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 获取所有图片文件
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = [f for f in os.listdir(image_dir) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    for image_file in image_files:
        # 获取图片路径和文件名（不含扩展名）
        image_path = os.path.join(image_dir, image_file)
        base_name = os.path.splitext(image_file)[0]
        
        # 对应的标签文件路径
        label_file = os.path.join(label_dir, f"{base_name}.txt")
        
        # 检查标签文件是否存在
        if not os.path.exists(label_file):
            print(f"警告: 未找到 {image_file} 对应的标签文件 {label_file}")
            continue
        
        # 读取图片
        image = cv2.imread(image_path)
        if image is None:
            print(f"警告: 无法读取图片 {image_file}")
            continue
        
        height, width = image.shape[:2]
        
        # 读取并解析标签文件
        with open(label_file, 'r') as f:
            lines = f.readlines()
        
        # 绘制每个目标的边界框
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # YOLO格式: class_id center_x center_y width height (均为归一化值)
            parts = line.split()
            if len(parts) != 5:
                print(f"警告: 标签文件 {label_file} 中格式错误: {line}")
                continue
            
            class_id = int(parts[0])
            center_x = float(parts[1])
            center_y = float(parts[2])
            bbox_width = float(parts[3])
            bbox_height = float(parts[4])
            
            # 转换为绝对坐标
            x = int((center_x - bbox_width / 2) * width)
            y = int((center_y - bbox_height / 2) * height)
            w = int(bbox_width * width)
            h = int(bbox_height * height)
            
            # 绘制边界框 - 修改为红色且线条更细
            # color = (0, 0, 255)  # 红色 (BGR格式，所以是0,0,255)
            color = (255, 0, 0) 
            # color = (0, 210, 210) 
            thickness = 1  # 线条粗细，改为1使边框更细
            
            if use_dashed_box:
                # 绘制密集小点的虚线框
                dot_length = 1    # 点的长度
                gap_length = 4    # 点之间的间隔
                # 上边
                for i in range(x, x + w, dot_length + gap_length):
                    end_x = min(i + dot_length, x + w)
                    cv2.line(image, (i, y), (end_x, y), color, thickness)
                # 下边
                for i in range(x, x + w, dot_length + gap_length):
                    end_x = min(i + dot_length, x + w)
                    cv2.line(image, (i, y + h), (end_x, y + h), color, thickness)
                # 左边
                for i in range(y, y + h, dot_length + gap_length):
                    end_y = min(i + dot_length, y + h)
                    cv2.line(image, (x, i), (x, end_y), color, thickness)
                # 右边
                for i in range(y, y + h, dot_length + gap_length):
                    end_y = min(i + dot_length, y + h)
                    cv2.line(image, (x + w, i), (x + w, end_y), color, thickness)
            else:
                # 绘制实线框
                cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
            
            # 绘制类别信息
            if class_names and class_id < len(class_names):
                label = f"{class_names[class_id]}"
            else:
                label = f"Class {class_id}"
            
            # # 放置标签文本（使用红色）
            # cv2.putText(image, label, (x, y - 10), 
            #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)  # 也减小了字体和线条粗细
        
        # 保存可视化结果
        output_path = os.path.join(output_dir, image_file)
        cv2.imwrite(output_path, image)
        print(f"已处理: {image_file} -> 保存到 {output_path}")

if __name__ == "__main__":
    # 配置路径（请根据实际情况修改）
    IMAGE_DIR = r"/home/gonghanmei/project/yolo/vis_predict"   # 图片所在文件夹
    LABEL_DIR = r"/home/gonghanmei/project/yolo/review_data_reg/labels/predict"# 标签所在文件夹
    # LABEL_DIR = r"/home/gonghanmei/datasets/sz_data_all/merged_folder/processed_labels"
    OUTPUT_DIR = r"/home/gonghanmei/project/yolo/vis_predict" # 可视化结果输出文件夹
    


    # 类别名称列表，如果有的话
    # 例如: CLASS_NAMES = ["class1", "class2", "class3"]
    CLASS_NAMES = None  # 如果没有类别名称列表，设为None
    
    # 执行可视化，设置use_dashed_box=True使用密集小点虚线框，False使用实线框
    visualize_yolo_labels(IMAGE_DIR, LABEL_DIR, OUTPUT_DIR, CLASS_NAMES, use_dashed_box=True)
    print("所有图片处理完成！")


In [ ]:
# 自适应裁剪
import os
import numpy as np
from PIL import Image
from pathlib import Path

def find_bottom_content(img_array, threshold=10):
    """
    找到图像底部有内容的最后一行
    
    参数:
        img_array: 图像的numpy数组
        threshold: 像素值阈值，大于此值认为是内容
    
    返回:
        最后一行有内容的行索引
    """
    height = img_array.shape[0]
    
    # 从底部向上扫描
    for i in range(height - 1, -1, -1):
        row = img_array[i]
        # 如果这一行有任何像素值大于阈值，认为是有内容的
        if np.any(row > threshold):
            return i
    
    return height - 1

def crop_bottom_black(input_path, output_path, bottom_margin=20, threshold=10):
    """
    裁剪图像底部的黑色区域，保留指定的底部边距
    
    参数:
        input_path: 输入图像路径
        output_path: 输出图像路径
        bottom_margin: 底部保留的像素数
        threshold: 判断是否为黑色的阈值
    """
    # 读取图像
    img = Image.open(input_path)
    
    # 转换为numpy数组
    img_array = np.array(img)
    
    # 处理不同的图像格式
    if len(img_array.shape) == 3:  # RGB或RGBA
        # 使用灰度值判断
        gray = np.mean(img_array[:, :, :3], axis=2)
    else:  # 灰度图
        gray = img_array
    
    # 找到底部最后有内容的行
    last_content_row = find_bottom_content(gray, threshold)
    
    # 计算裁剪位置（保留bottom_margin像素）
    crop_bottom = min(last_content_row + bottom_margin, img_array.shape[0])
    
    # 裁剪图像
    cropped_array = img_array[:crop_bottom, :]
    
    # 转换回PIL图像并保存
    cropped_img = Image.fromarray(cropped_array)
    cropped_img.save(output_path)
    
    return crop_bottom

def batch_crop_images(input_folder, output_folder=None, bottom_margin=20, threshold=10):
    """
    批量处理文件夹中的所有PNG图像
    
    参数:
        input_folder: 输入文件夹路径
        output_folder: 输出文件夹路径（如果为None，则在输入文件夹创建cropped子文件夹）
        bottom_margin: 底部保留的像素数
        threshold: 判断是否为黑色的阈值
    """
    input_path = Path(input_folder)
    
    # 设置输出文件夹
    if output_folder is None:
        output_path = input_path / "cropped"
    else:
        output_path = Path(output_folder)
    
    # 创建输出文件夹
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 获取所有PNG文件
    png_files = list(input_path.glob("*.png")) + list(input_path.glob("*.PNG"))
    
    if not png_files:
        print(f"在 {input_folder} 中没有找到PNG文件")
        return
    
    print(f"找到 {len(png_files)} 个PNG文件")
    print(f"开始处理，底部保留 {bottom_margin} 像素...")
    
    # 处理每个文件
    for i, png_file in enumerate(png_files, 1):
        try:
            output_file = output_path / png_file.name
            crop_height = crop_bottom_black(
                str(png_file), 
                str(output_file), 
                bottom_margin=bottom_margin,
                threshold=threshold
            )
            print(f"[{i}/{len(png_files)}] {png_file.name} - 裁剪至高度 {crop_height}")
        except Exception as e:
            print(f"[{i}/{len(png_files)}] 处理 {png_file.name} 时出错: {e}")
    
    print(f"\n完成! 处理后的图像保存在: {output_path}")

# 使用示例
if __name__ == "__main__":
    # 设置你的文件夹路径
    input_folder = "/home/gonghanmei/project/yolo/vis_predict"
    
    # 方式1: 自动创建cropped子文件夹
    batch_crop_images(input_folder, bottom_margin=20)
    
    # 方式2: 指定输出文件夹
    # output_folder = "/home/gonghanmei/datasets/shen_data/png-all-jq/vis_cropped"
    # batch_crop_images(input_folder, output_folder, bottom_margin=20)
    
    # 如果需要调整阈值（默认10），可以这样：
    batch_crop_images(input_folder, bottom_margin=20, threshold=15)


In [ ]:
# 提取分割可视化
import os
import glob
import shutil
from datetime import datetime

def collect_compare_images(search_root='seg_output_full5', collection_dir='compare_images_collection6'):
    """
    递归查找指定目录下所有以 '_compare.png' 结尾的文件，并将它们复制到目标集合目录。

    Args:
        search_root (str): 开始搜索的根目录。
        collection_dir (str): 复制文件的目标目录。
    """
    
    print(f"--- 图像提取任务启动 ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')}) ---")
    print(f"🔍 搜索目录: {os.path.abspath(search_root)}")
    
    # 1. 确保目标目录存在
    try:
        os.makedirs(collection_dir, exist_ok=True)
        print(f"📂 目标目录已准备: {os.path.abspath(collection_dir)}")
    except OSError as e:
        print(f"❌ 错误：无法创建目标目录 {collection_dir}。权限或路径问题。")
        print(f"错误信息: {e}")
        return

    # 2. 定义搜索模式并执行递归查找
    # '**/*' 表示递归地进入所有子目录
    file_pattern = os.path.join(search_root, '**/*_compare.png')
    
    # 使用 glob.glob 和 recursive=True 进行查找
    compare_files = glob.glob(file_pattern, recursive=True)

    if not compare_files:
        print("\n⚠️ 未找到任何匹配 '_compare.png' 后缀的文件。请检查路径和文件名。")
        return

    print(f"\n✅ 找到 {len(compare_files)} 个匹配文件，开始复制...")
    
    copied_count = 0
    
    # 3. 遍历并复制文件
    for src_path in compare_files:
        # 获取文件名 (例如: 2024-10-2508-PengMeiYun_front_compare.png)
        file_name = os.path.basename(src_path)
        
        # 构造目标文件的完整路径
        dst_path = os.path.join(collection_dir, file_name)
        
        # 为了避免文件名冲突，我们可以在文件名中添加一个唯一的ID或目录名
        # 比如：将其父目录名作为前缀（例如：2024-10-2508-PengMeiYun_front_compare.png -> PengMeiYun_front_2024-10-2508-PengMeiYun_front_compare.png）
        # 鉴于您的输入路径结构，文件重名可能性较大，我们采用更稳健的重命名策略：
        if os.path.exists(dst_path):
            # 获取上级目录名，例如 '2024-10-2508-PengMeiYun_front'
            parent_dir_name = os.path.basename(os.path.dirname(src_path))
            
            # 创建新的文件名：父目录名_文件名
            new_file_name = f"{parent_dir_name}_{file_name}"
            dst_path = os.path.join(collection_dir, new_file_name)
            
            print(f"   * 重命名：{file_name} -> {new_file_name}")

        try:
            # 使用 shutil.copy2 复制文件，它会保留文件的元数据（如创建时间）
            shutil.copy2(src_path, dst_path)
            # print(f"   - 复制成功：{src_path}") # 调试信息，可以取消注释
            copied_count += 1
        except Exception as e:
            print(f"   ❌ 复制失败：{src_path} 到 {dst_path}")
            print(f"   错误信息: {e}")

    print(f"\n--- 任务完成 ---")
    print(f"✨ 成功复制 {copied_count} 个文件到 {collection_dir} 目录。")


# 运行函数
if __name__ == "__main__":
    # 请确保您在运行脚本时，'seg_output_full' 目录位于当前工作目录下，
    # 或者将 search_root 设置为绝对路径。
    collect_compare_images()
